In [1]:
import nanopq
import numpy as np
import networkx as nx
from collections import defaultdict
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_blobs
import sys
import faiss

In [2]:
N, D = 1000000, 256

CLUSTERS_NUMBER = 10000  # число групп

np.random.seed(0)
centers = np.random.randint(-2000, 2000, size=(CLUSTERS_NUMBER, D))  # создает K центров D-размерности, расположенных случайно

vectors, y = make_blobs(n_samples=N, n_features=D, centers=centers, random_state=0)
vectors = vectors.astype(np.float32)

vectors_base, train_y = vectors[:N-10000], y[:N-10000]
queries, test_y = vectors[N-10000:], y[N-10000:]

In [3]:
N, D = 1000000, 128

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()


def fvecs_read(fname):
    return ivecs_read(fname).view('float32')

def load_sift():
    print("Loading sift...", end='', file=sys.stderr)
    xb = fvecs_read("sift/sift_base.fvecs")
    xq = fvecs_read("sift/sift_query.fvecs")
    gt = ivecs_read("sift/sift_groundtruth.ivecs")
    print("done", file=sys.stderr)

    return xb, xq, gt

def load_gist():
    print("Loading gist...", end='', file=sys.stderr)
    xb = fvecs_read("gist/gist_base.fvecs")
    xq = fvecs_read("gist/gist_query.fvecs")
    gt = ivecs_read("gist/gist_groundtruth.ivecs")
    print("done", file=sys.stderr)

    return xb, xq, gt


vectors_base, queries, gt = load_sift()

Loading sift...done


In [16]:
   
def generate_graph(vectors, k_nearest):
    index = faiss.IndexFlatL2(vectors.shape[1])  # длина вектора
    index.add(vectors.astype('float32'))
    _, indices = index.search(vectors.astype('float32'), k_nearest)
    G = nx.Graph()
    for i in tqdm(range(len(vectors)), total=len(vectors)):
        for index in indices[i]:
            if index != i:
                G.add_edge(i, index)
    return G


k = 35    # количество ближайших соседей для связывания вершин


# генерируем граф
G = generate_graph(vectors_base, k)


100%|██████████| 990000/990000 [00:46<00:00, 21211.91it/s]


In [22]:
pq = nanopq.PQ(M=2, Ks=128, verbose=False)

pq.fit(vectors_base)
X_code = pq.encode(vectors_base)

In [23]:
indexes_map = defaultdict(list)

for i in range(len(X_code)):
    indexes_map[(X_code[i][0], X_code[i][1])].append(i)

In [24]:
def find_nearest(G, query, indexes_map=indexes_map, pq=pq, vectors=vectors_base):
    query_mi = pq.encode(query)
    query_mi_neighbours = indexes_map[(query_mi[0][0], query_mi[0][1])]
    if len(query_mi_neighbours) > 0:
        best_node = query_mi_neighbours[0]
    else:
        best_node = np.random.choice(G.nodes())
    best_dist = np.linalg.norm(vectors[best_node] - query)
    queue = []
    queue.append(best_node)
    was = set()
    was.add(best_node)
    while len(queue) > 0:
        node = queue.pop(0)
        for edge in G.edges(node):
            dst = edge[1]
            if dst in was:
                continue
            was.add(dst)
            dist = np.linalg.norm(vectors[dst] - query)
            if dist < best_dist:
                queue.append(dst)
                best_node = dst
                best_dist = dist
    return best_dist, best_node


In [10]:
def find_nearest_fi(G, query, indexes_map=indexes_map, pq=pq, vectors=vectors_base):
    query_mi = pq.encode(query)
    query_mi_neighbours = indexes_map[(query_mi[0][0], query_mi[0][1])]
    if len(query_mi_neighbours) > 0:
        best_node = query_mi_neighbours[0]
    else:
        best_node = np.random.choice(G.nodes())
    best_dist = np.linalg.norm(vectors[best_node] - query)
    was = set()
    was.add(best_node)
    while True:
        node = best_node
        edges = list(G.edges(node))
        for edge_id in range(len(edges)-1, 0, -1):
            edge = edges[edge_id]
            dst = edge[1]
            if dst in was:
                continue
            was.add(node)
            dist = np.linalg.norm(vectors[dst] - query)
            if dist < best_dist:
                best_node = dst
                best_dist = dist
                break
        if best_node == node:
            break
    return best_dist, best_node


In [11]:
def find_nearest_v2(G, query, indexes_map=indexes_map, pq=pq, vectors=vectors_base):
    query_mi = pq.encode(query)
    query_mi_neighbours = indexes_map[(query_mi[0][0], query_mi[0][1])]
    if len(query_mi_neighbours) > 0:
        best_node = query_mi_neighbours[0]
        best_dist = np.linalg.norm(vectors[best_node] - query)
        for best_candidate in query_mi_neighbours:
            dist = np.linalg.norm(vectors[best_candidate] - query)
            if dist < best_dist:
                best_node = best_candidate
                best_dist = dist
    else:
        best_node = np.random.choice(G.nodes())
        best_dist = np.linalg.norm(vectors[best_node] - query)
    queue = []
    queue.append(best_node)
    was = set()
    was.add(best_node)
    while len(queue) > 0:
        node = queue.pop(0)
        for edge in G.edges(node):
            dst = edge[1]
            if dst in was:
                continue
            was.add(node)
            dist = np.linalg.norm(vectors[dst] - query)
            if dist < best_dist:
                queue.append(dst)
                best_node = dst
                best_dist = dist
    return best_dist, best_node


In [25]:
index = faiss.IndexHNSWFlat(D, 32)
index.hnsw.efConstruction = 40

index.train(vectors_base)
print(index.ntotal)   # 0
index.add(vectors_base)
print(index.ntotal)   # 1000000

0
990000


In [28]:
import time

dist_better_my = 0
dist_better_hnsw = 0
dist_equal = 0

gt_good_my = 0
gt_good_hnsw = 0

result_my = []
result_hnsw = []

start = time.time()

was = defaultdict(int)

for test_number in tqdm(range(len(queries))):
    d1, i1 = find_nearest(G, np.array([queries[test_number]]))
    result_my.append([d1, i1])

end = time.time()

print("my time:", end-start)
start = time.time()
for test_number in range(len(queries)):
    d2, i2 = index.search(np.array([queries[test_number]]), 1)
    d2 = np.linalg.norm(vectors_base[i2[0][0]] - queries[test_number])
    result_hnsw.append([d2, i2[0][0]])
end = time.time()
print("hnsw time:", end-start)

for test_number in range(len(queries)):
    d1, i1 = result_my[test_number][0], result_my[test_number][1]
    gt_good_my += test_y[test_number] == train_y[i1]
    
    d2, i2 = result_hnsw[test_number][0], result_hnsw[test_number][1]
    gt_good_hnsw += test_y[test_number] == train_y[i2]
    
    dist_better_my += d1 < d2
    dist_equal += d1 == d2
    dist_better_hnsw += d1 > d2
print(dist_better_my, dist_equal, dist_better_hnsw)
print(gt_good_my, gt_good_hnsw)

100%|██████████| 10000/10000 [00:03<00:00, 2812.09it/s]


my time: 3.5575976371765137
hnsw time: 8.984891414642334
952 5764 3284
7017 9666


In [13]:
test_y[0]

6973

my data

base

my time: 3.3900723457336426

hnsw time: 2.715017557144165

895 6261 2844

7340 9705

In [164]:
T = 21
print(find_nearest(G, np.array([queries[T]])))
print(index.search(np.array([queries[T]]), 1)[1][0][0])
print(gt[T])

(173.01733, 337194)
4490
[  4490   4457 440896 105025  30046 214449 214475 234950 337194 566609
 554601 559990 497354 337027 306259  59773 365266 776961 261517  30205
 498443 104882 293784 787655 187642 161744  94526 290612 190535 497211
 104971 554692 321611 554617 523702  99066 151645 501915 710368 525244
 261426 570662  94528 114300 278717 190529 366228 337168 365353 312454
 776967 825736 563905 547063 357803  51303 122700 805687 776965 152427
 498491 869260 124074 106313 534926  42882 337012 122254 494271 825636
  94532 151174 428246 790898  51399 365529 674780 365285 560050  52746
 526223 295102 122033 292386 151148 337470 689876 776959 385048 302199
 525423 160586 427618 852710 152142 905096  90283 571084 305080 114777]


In [165]:
pq.encode(np.array([queries[T]]))


array([[85,  9]], dtype=uint8)

In [166]:
pq.encode(np.array([vectors_base[337194]]))


array([[85,  9]], dtype=uint8)

In [167]:
pq.encode(np.array([vectors_base[4490]]))


array([[24,  9]], dtype=uint8)